In [1]:
!pip install silence_tensorflow
!conda install -y -c conda-forge fbprophet

Solving environment: ...working... done

# All requested packages already installed.



In [1]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from fbprophet import Prophet

In [2]:
def plot(data):
    splot = go.Figure()
    splot.add_trace(go.Scatter(x=data.index, y=data.values, name='Cazuri confirmate'))
    splot.update_layout(xaxis_title='Zile', yaxis_title='Nr cazuri', title='Cazuri confirmate zilnice la 1000 loc')
    splot.show()

In [3]:
csv = pd.read_csv(os.path.join(os.path.curdir, 'kaggle', 'covid19', 'input', 'time_series_covid_19_confirmed.csv'), na_values=['?', 'na', 'nan', 'NaN', 'Unknown'])
csv[['Province/State']] = csv[['Province/State']].fillna('Unknown')
csv.drop(['Lat', 'Long'], axis=1, inplace=True)
csv.set_index(['Province/State','Country/Region'], inplace=True)
romania = csv.loc[('Unknown', 'Romania'),]
romania = romania.diff().fillna(romania[0])/1000
plot(romania)
romania = pd.DataFrame({'ds': pd.to_datetime(romania.index), 'y': romania.values}, columns=['ds', 'y'])

In [4]:
model = Prophet(daily_seasonality=False, yearly_seasonality=False)
nrTotal = len(romania['ds'])
test = romania.loc[nrTotal-28:,]
train = romania.loc[:nrTotal-28,]
model.fit(train)
print(str(test[['ds']].min()['ds']).split(' ')[0], '-', str(test[['ds']].max()['ds']).split(' ')[0])
future = model.make_future_dataframe(periods=27)
future.tail()

2020-11-09 - 2020-12-06


,ds
315,2020-12-02
316,2020-12-03
317,2020-12-04
318,2020-12-05
319,2020-12-06


In [5]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
315,2020-12-02,8.151859,7.345317,8.974615
316,2020-12-03,8.302122,7.443840,9.083118
317,2020-12-04,8.373943,7.542498,9.146496
318,2020-12-05,8.390079,7.589673,9.219550
319,2020-12-06,8.268776,7.488174,9.069911


In [6]:
from tensorflow.keras.losses import mean_squared_error
loss = np.sqrt(mean_squared_error(romania['y'], forecast['yhat']))
print(f'Scor predictie (RMSE): {loss}')

Scor predictie (RMSE): 0.8963779230583288


In [7]:
from fbprophet.plot import plot_plotly
plot_plotly(model, forecast)